In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, mean_squared_error
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, LassoCV, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, mean_squared_error

In [ ]:
##filter warnings
import warnings
warnings.filterwarnings('ignore')


In [ ]:
housing_df = pd.read_csv(r'D:\Data Analyst Boot Camp\Assignment Data\Assignment 5\Ames Dataset\ames_data_no_missing.csv')
housing_df

In [ ]:
# What is the distibution of housing prices?
#Given the deistribution, would it be appropriate to use a single model to predict sales for houses or, should we subset that data?
#Are there any variable transformations that would be potentially useful?

housing_df.hist(bins=50, figsize = (20,22))
plt.show()

#With the distribution it would best to subset the data to get a better prediction.

In [ ]:
housing_df.info()

In [ ]:
#Which variable has the highest correlation with housing prices?
#What are the key drivers behinid larger housing prices?
corr_matrix = housing_df.corr()
corr_matrix

In [ ]:
corr_matrix["SalePrice"].sort_values(ascending = False)
#The Overall quality of the house tends to be the driver of the larger housing price.

In [ ]:
#Create one additional visualization that provides some insights into the data.

sns.catplot(x='Yr Sold', y= 'SalePrice', kind = 'box', data = housing_df)

In [ ]:
#Prepare the data for the machine learning model
non_categorical_columns = ['Overall Qual', 'Overall Cond','Year Built']

for column in housing_df.columns:
    if column not in non_categorical_columns:
        housing_df[column] = pd.Categorical(housing_df[column])

In [ ]:
#Store the predictors and the target variable in variables x and y
y = housing_df['SalePrice']
x = housing_df.iloc[:,:-2]

In [ ]:
#Create the dummy variables for categorical columns
x = pd.get_dummies(x, drop_first = False)

In [ ]:
#Split the data into train and test sets (70-30 split with random state 0)
x_train, x_val, y_train, y_val = train_test_split(x,y, test_size = 0.3, random_state =0)

In [ ]:
#Scale the numerical columns using StandardScaler.
scaler = StandardScaler()
x_train[['Overall Qual', 'Overall Cond','Year Built']] = scaler.fit_transform(x_train[['Overall Qual', 'Overall Cond','Year Built']])
x_val[['Overall Qual', 'Overall Cond','Year Built']] = scaler.transform(x_val[['Overall Qual', 'Overall Cond','Year Built']])

In [ ]:
#Build a linear regression model, A Regression Tree, and a k-NN model.

In [ ]:
#Linear Regression Model

lin_reg = LinearRegression()
lin_reg.fit(x,y)
lin_reg.intercept_, lin_reg.coef_
lin_reg.predict(x)

In [ ]:
model = sm.OLS(y,x)
results = model.fit()
results.params
results.summary()
print(results.summary())

In [ ]:
sns.relplot(x='Overall Qual', y ='SalePrice', data = housing_df,
            height = 3.8, kind = 'scatter')
sns.set_style('dark')

In [ ]:
sns.regplot('Overall Qual', 'SalePrice', data = housing_df)
plt.title('Overall Quality determines SalePrice')

In [ ]:
# Regression Tree
tree_clf = DecisionTreeClassifier(random_state = 0, max_depth = 10)
tree_clf.fit(x_train, y_train)

In [ ]:
fig = plt.figure(figsize = (16,8))
fig = tree.plot_tree(tree_clf, feature_names = x.columns, filled = True)

In [ ]:
# k-NN Model
kvalues = np.arange(1,30)
val_acc = []
for k in kvalues:
    knn_clf = KNeighborsClassifier(n_neighbors = k)
    knn_clf.fit(x_train, y_train)
    y_pred = knn_clf.predict(x_val)
    val_acc.append(accuracy_score(y_val, y_pred))

plt.plot(kvalues, val_acc, marker='o')
plt.xlabel("k")
plt.ylabel("SalePrice")

print("The highest SalePrice is at K=" + str(np.argmax(val_acc)+1))